In [1]:
import os
import zipfile
import subprocess
import pandas as pd
from datetime import date, timedelta
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

In [2]:


def get_latest_master():
    subprocess.run(["kaggle", "datasets", "download", "-d", "benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated", "-p", "data"])
    #!kaggle datasets download -d benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated -p data
    zip_file_path = "data/s-and-p-500-with-dividends-and-splits-daily-updated.zip"
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall("data")

    df = pd.read_csv("archive/sp500_stocks.csv")
    sp = df["Symbol"].unique()
    sp.sort()
    sp = list(sp)

    data = os.listdir("data")
    data.sort()
    data = list(data)

    # create master dataframe from single dfs
    df = pd.DataFrame()
    for i in data:
        if i.replace(".csv", "") in sp:
            df2 = pd.read_csv(f"data/{i}")
            df2["company"] = i.replace(".csv", "")
            df = pd.concat([df, df2], ignore_index=True)

    df.to_csv("master1.csv")
    return df





In [ ]:

    # script creates dataframe for ML
    import create_ML1_df
    ml1 = create_ML1_df(master1)
    ml1.to_csv("ml1")

    import create_ML1_df_training
    ml2 = create_ML1_df_training.create_ML1_df_training()
    ml2.to_csv("ml1_training")


    , "Unnamed: 0"


In [3]:
def create_ML1_df(df):

    # import data
    #df = pd.read_csv("master1.csv")
    caps_df = pd.read_csv("caps_df.csv")
    sector_df = pd.read_csv("sector_df.csv")

    # add information of marketcap and (sub)sectors
    df = df.merge(caps_df, how="inner", on="company")
    df = df.merge(sector_df, how="inner", on="company")
    df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y"], inplace=True)

    # data cleaning
    df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]
    df["date"] = pd.to_datetime(df["date"], utc=True)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df["marketcap"] = df["marketcap"].map(lambda x: x.replace("B", "").replace(",", ""))
    df["marketcap"] = df["marketcap"].map(lambda x: pd.to_numeric(x))

    # moving average, bollinger bands, price change, RSI
    grouped_df = df.groupby("company")

    def ma_bb(company):
        company = company.sort_values(by="date").set_index("date")

        # indicators
        company["ma_20"] = company["close"].rolling(window=20).mean()
        company["ma_60"] = company["close"].rolling(window=60).mean()
        company["bb_lower"] = company["close"].rolling(window=20).mean() - company["close"].rolling(window=20).std()*2
        company["bb_upper"] = company["close"].rolling(window=20).mean() + company["close"].rolling(window=20).std()*2

        company["price_change"] = company["close"].diff()

        company["gain_14"] = company["price_change"].clip(lower=0).rolling(window=14).mean()
        company["loss_14"] = company["price_change"].clip(upper=0).rolling(window=14).mean()

        company["rsi"] = 100 - (100 / (1 + (company["gain_14"] / abs(company["loss_14"]))))
        company["rsi_14"] = company["rsi"].rolling(window=14).mean()

        company["ema_12"] = company["close"].ewm(span=12, adjust=False).mean()
        company["ema_26"] = company["close"].ewm(span=26, adjust=False).mean()
        company["macd"] = (company["ema_12"] - company["ema_26"]).ewm(span=9, adjust=False).mean()


        # lagged indicators
        company["rsi_lag_5"] = company["rsi_14"].shift(5)
        company["rsi_lag_10"] = company["rsi_14"].shift(10)

        # target
        #company["price_30d"] = company["close"].shift(-30)

        return company

    df2 = grouped_df.apply(ma_bb).dropna()

    # dropping not needed price features
    df3 = df2.copy()
    df3.drop(columns=["open", "high", "low"], inplace=True)

    df3.drop(columns="company", inplace=True)
    df3.reset_index(inplace=True)
    df3.set_index("date", inplace=True)

    df3.to_csv("ml1.csv")
    return df3


In [4]:
def create_ML1_df_training(df):

    # import data
    #df = pd.read_csv("master1.csv")
    caps_df = pd.read_csv("caps_df.csv")
    sector_df = pd.read_csv("sector_df.csv")

    # add information of marketcap and (sub)sectors
    df = df.merge(caps_df, how="inner", on="company")
    df = df.merge(sector_df, how="inner", on="company")
    df.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y"], inplace=True)

    # data cleaning
    df.columns = [columns.lower().replace(" ", "_") for columns in df.columns]
    df["date"] = pd.to_datetime(df["date"], utc=True)
    df["date"] = pd.to_datetime(df["date"]).dt.date
    df["marketcap"] = df["marketcap"].map(lambda x: x.replace("B", "").replace(",", ""))
    df["marketcap"] = df["marketcap"].map(lambda x: pd.to_numeric(x))

    # moving average, bollinger bands, price change, RSI
    grouped_df = df.groupby("company")

    def ma_bb(company):
        company = company.sort_values(by="date").set_index("date")

        # indicators
        company["ma_20"] = company["close"].rolling(window=20).mean()
        company["ma_60"] = company["close"].rolling(window=60).mean()
        company["bb_lower"] = company["close"].rolling(window=20).mean() - company["close"].rolling(window=20).std()*2
        company["bb_upper"] = company["close"].rolling(window=20).mean() + company["close"].rolling(window=20).std()*2

        company["price_change"] = company["close"].diff()

        company["gain_14"] = company["price_change"].clip(lower=0).rolling(window=14).mean()
        company["loss_14"] = company["price_change"].clip(upper=0).rolling(window=14).mean()

        company["rsi"] = 100 - (100 / (1 + (company["gain_14"] / abs(company["loss_14"]))))
        company["rsi_14"] = company["rsi"].rolling(window=14).mean()

        company["ema_12"] = company["close"].ewm(span=12, adjust=False).mean()
        company["ema_26"] = company["close"].ewm(span=26, adjust=False).mean()
        company["macd"] = (company["ema_12"] - company["ema_26"]).ewm(span=9, adjust=False).mean()


        # lagged indicators
        company["rsi_lag_5"] = company["rsi_14"].shift(5)
        company["rsi_lag_10"] = company["rsi_14"].shift(10)

        # target
        company["price_30d"] = company["close"].shift(-30)

        return company

    df2 = grouped_df.apply(ma_bb).dropna()

    # dropping not needed price features
    df3 = df2.copy()
    df3.drop(columns=["open", "high", "low"], inplace=True)

    df3.drop(columns="company", inplace=True)
    df3.reset_index(inplace=True)
    df3.set_index("date", inplace=True)

    df3.to_csv("ml1_training.csv")

    return df3


In [5]:
def run_xgb(df4, df4_training):

    def XGB_train_real():

        # short cleaning
        df_x = df4_training.copy().reset_index()
        df_x.drop(columns=["company", "sector", "subsector"], inplace=True)
        df_x.set_index("date", inplace=True)
        df_x.sort_index(inplace=True)

        # training with data only until training_end
        xg_df = df_x.copy()

        # X y split
        X = xg_df.drop(columns="price_30d")
        y = xg_df["price_30d"]

        # normalization
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)

        # train model
        xgbr = xgb.XGBRFRegressor()
        xgbr.fit(X_scaled, y)

        return xgbr



    def get_stocks():

        """
        Output: df with top 20 gainers
        """

        yesterday = date.today() - timedelta(days=2)
        df_ga = df4.copy()
        df_ga.reset_index(inplace=True)
        df_ga["date"] = pd.to_datetime(df_ga["date"])
        gains_df = pd.DataFrame(columns=["company", "close", "prediction", "gain_predicted"])
        df_ga.drop(columns=["sector", "subsector"], inplace=True)
        df_ga.set_index("date", inplace=True)    
        df_ga.sort_index(inplace=True)

        xg_df = df_ga.loc[str(yesterday)]
        xg_df.sort_index(inplace=True)

        X = xg_df.drop(columns=["company"])
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        pred_xgb = xgbr.predict(X_scaled)
        test = xg_df[["close", "company"]]
        test["prediction"] = pred_xgb
        test["gain_predicted"] = (test["prediction"] - test["close"]) / test["close"] * 100
        test.sort_values(by="gain_predicted", ascending=False, inplace=True)
        #gain = test.head(10)["gain_real"].mean()

        return test

    # import data
    #df4 = pd.read_csv("ml1.csv")
    #df4_training = pd.read_csv("ml1_training.csv")

    

    xgbr = XGB_train_real()
    stocks = get_stocks()
    return stocks.head(20)

In [6]:
step1 = get_latest_master()

Dataset URL: https://www.kaggle.com/datasets/benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated
License(s): CC-BY-SA-4.0
... resuming from 338467800 bytes (-8305499 bytes left) ...
416 - Requested range not satisfiable


In [7]:
ml1 = create_ML1_df(step1)

/var/folders/4d/kvxdy8f57vb9008_7qx5gx780000gn/T/ipykernel_19142/1842159998.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = grouped_df.apply(ma_bb).dropna()


In [8]:
ml_training = create_ML1_df_training(step1)

/var/folders/4d/kvxdy8f57vb9008_7qx5gx780000gn/T/ipykernel_19142/542916464.py:54: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df2 = grouped_df.apply(ma_bb).dropna()


In [9]:
tipps = run_xgb(ml1, ml_training)

/var/folders/4d/kvxdy8f57vb9008_7qx5gx780000gn/T/ipykernel_19142/3832922460.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["prediction"] = pred_xgb
/var/folders/4d/kvxdy8f57vb9008_7qx5gx780000gn/T/ipykernel_19142/3832922460.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["gain_predicted"] = (test["prediction"] - test["close"]) / test["close"] * 100
/var/folders/4d/kvxdy8f57vb9008_7qx5gx780000gn/T/ipykernel_19142/3832922460.py:55: SettingWithCopyWarning: 
A value is trying to be set on a